In [1]:
%load_ext autoreload
%autoreload 2

### imports and utilities

In [2]:
!pip install nltk scikit-learn matplotlib scipy -q

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>


False

In [49]:
from collections import Counter
from utils import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from scipy.special import softmax
from scipy.stats import norm
from scipy.stats import entropy as calculate_entropy

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### load dataset

In [50]:
dataset = "newsgroup"

# total number of samples needed
randomize = False

# retrieve dataset
categories = ['rec.autos', 'talk.politics.mideast', 'alt.atheism', 'sci.space']

all_docs = fetch_20newsgroups(subset='train', shuffle=randomize, remove=('headers', 'footers', 'quotes'), categories=categories)
all_docs, old_labels, categories = all_docs.data, all_docs.target, all_docs.target_names

In [51]:
# dataset = "bbc"

# data = pd.read_csv('bbcsport.csv')

# all_docs = data["text"].to_list()
# old_labels = data["topic"].to_list()
# categories = classes = np.unique(data["topic"]).tolist()

### clean dataset

In [56]:
datasize = 40
min_document_length = 160
max_document_length = 256


index = -1
docs, labels, label_indices = [], [], []

sizes = [0]*len(categories)

with tqdm(total=len(categories)*datasize) as pbar:
    while sum(sizes) < len(categories)*datasize:
        index += 1
        label_index = old_labels[index]
            
        if sizes[label_index] == datasize:
            continue
        
        doc = all_docs[index]
        status, doc, word_count = clean_doc(doc, True)
        
        if not status:
            continue
            
        if min_document_length is not None and len(doc) < min_document_length:
            continue
            
        if max_document_length is not None and len(doc) > max_document_length:
            continue
        
        label_indices.append(label_index)
        labels.append(categories[label_index])
        
        docs.append(doc)
        sizes[label_index] += 1
        pbar.update(1)

labels = np.array(labels)
label_indices = np.array(label_indices)

In [57]:
doc_index = 3
print(f"Topic: {labels[doc_index]}\n{'='*50}\n{docs[doc_index][:512]}")

Topic: rec.autos
not to mention my friend s 54 citroen traction avant with the light switch and dimmer integrate in a single stalk off the steer column those dumb french be apparently copying the japanese before the german


In [58]:
print(sizes)
assert min(sizes) == max(sizes) == datasize

[40, 40, 40, 40]


### Split data

In [59]:
x_train, x_test, y_train, y_test = train_test_split(docs, labels, test_size =.3)

In [60]:
print(f"there are {len(docs)} total docs, {len(y_train)} train and {len(y_test)} test")

there are 160 total docs, 112 train and 48 test


### Initialize Vectorizer

In [61]:
# initialize the count vectorizer
vectorizer = CountVectorizer()

# fit it to dataset
vectorizer.fit(x_train)

vocabulary = np.array(vectorizer.get_feature_names())
print("word_count is", len(vocabulary))

word_count is 1449


### Prepare Datatset

In [62]:
# create doc count vectors
train_doc_vectors = vectorizer.transform(x_train).toarray()
test_doc_vectors = vectorizer.transform(x_test).toarray()

wdf_train = pd.DataFrame(train_doc_vectors, columns=vocabulary)
wdf_test = pd.DataFrame(test_doc_vectors, columns=vocabulary)

## Word Word Probability Distr

In [63]:
wdt_train = wdf_train.copy()
wdt_test = wdf_test.copy()

wdt_test["__labels__"] = y_test
wdt_train["__labels__"] = y_train

word_doc_count = wdf_train.sum(0)
word_word_pr_distr = pd.DataFrame(data=0.0, columns=vocabulary, index=vocabulary)

for word in tqdm(vocabulary):
    pxy = wdf_train[wdf_train[word] > 0].sum(0) / word_doc_count[word]
#     pxy = (wdf_train.T * wdf_train[word]).sum(1)/ word_doc_count[word]
    word_word_pr_distr[word] = pxy * (word_doc_count[word] / word_doc_count)

word_word_pr_distr /= word_word_pr_distr.max().max()
print(f"word_word_pr_distr shape = {word_word_pr_distr.shape}")


word_word_pr_distr shape = (1449, 1449)


In [64]:
word_word_pr_distr.head()

,000,031349,10,11,12cyl,1900,1940,1984,1987,1990,...,wrong,yeah,year,yes,yo,yorker,you,your,yourself,zeuge
000,1.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0
031349,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
10,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
11,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12cyl,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
word = "space"

topwords_df = word_word_pr_distr.loc[word].sort_values(ascending=False).head(10)
topwords = topwords_df.index.tolist()

print(f"Topwords: Word - {word}\n{'='*50}")
print(topwords_df)

Topwords: Word - space
space    1.0
be       1.0
to       0.8
if       0.8
from     0.8
them     0.8
on       0.6
have     0.6
and      0.6
the      0.6
Name: space, dtype: float64


### Modify Word Word Distr with entopy

In [66]:
word_word_pr_entropy = calculate_entropy(word_word_pr_distr, axis=1)
word_word_pr_entropy_softmax = softmax(word_word_pr_entropy.max() - word_word_pr_entropy)

In [67]:
word_word_pr_distr_prime = word_word_pr_distr.copy()

for word in tqdm(word_word_pr_distr.columns):
    word_word_pr_distr_prime.loc[word] *= word_word_pr_entropy_softmax

word_word_pr_distr_prime /= word_word_pr_distr_prime.max().max()

In [68]:
word = "space"

topwords_prime_df = word_word_pr_distr_prime.loc[word].sort_values(ascending=False).head(10)
topwords_prime = topwords_prime_df.index.tolist()

print(f"Topwords Prime: Word - {word}\n{'='*50}")
print(topwords_prime_df)

Topwords Prime: Word - space
space        0.325554
raw          0.311111
effective    0.311111
colapse      0.311111
material     0.311111
cost         0.311111
entire       0.311111
mode         0.247059
act          0.247059
activate     0.247059
Name: space, dtype: float64


### Compare Word Word Distr with Modified Word Word Distr

In [69]:
base_word = "space"
li = {
    "against self": [base_word],
    "against topwords" : topwords, 
    "against topwords prime": topwords_prime
}

for title, other_words in li.items():
    print(f'{" ".join([base_word, title]):^33s}')
    print("===============================")
    print("word            | wwpd  | wwpd_prime")
    print("-------------------------------")
    
    for other_word in other_words:
        r1 = word_word_pr_distr.loc[base_word][other_word] / word_word_pr_distr.loc[base_word].max()
        r2 = word_word_pr_distr_prime.loc[base_word][other_word] / word_word_pr_distr_prime.loc[base_word].max()

        print(f"{other_word:15s} | {r1:5.2f} | {r2:5.2f}")
    print()

       space against self        
word            | wwpd  | wwpd_prime
-------------------------------
space           |  1.00 |  1.00

     space against topwords      
word            | wwpd  | wwpd_prime
-------------------------------
space           |  1.00 |  1.00
be              |  1.00 |  0.13
to              |  0.80 |  0.11
if              |  0.80 |  0.16
from            |  0.80 |  0.23
them            |  0.80 |  0.35
on              |  0.60 |  0.11
have            |  0.60 |  0.11
and             |  0.60 |  0.08
the             |  0.60 |  0.07

  space against topwords prime   
word            | wwpd  | wwpd_prime
-------------------------------
space           |  1.00 |  1.00
raw             |  0.40 |  0.96
effective       |  0.40 |  0.96
colapse         |  0.40 |  0.96
material        |  0.40 |  0.96
cost            |  0.40 |  0.96
entire          |  0.40 |  0.96
mode            |  0.40 |  0.76
act             |  0.40 |  0.76
activate        |  0.40 |  0.76



### Document Topwords

In [70]:
doc_index = 0
print(f"Topic: {y_train[doc_index]}\n{'='*50}\n{x_train[doc_index][:512]}")

Topic: alt.atheism
sure but robert koresh fetesh sic knowles seem good too though i wasn t there at least i can rely on you now to keep me post on what what he s do have you any other fetish besides those for beef jerky and david koresh bake timmons iii


In [71]:
wdf_train_original = wdf_train / wdf_train.max().max()
wdf_test_original = wdf_test / wdf_test.max().max()
 
topwords_df = wdf_train_original.loc[doc_index].sort_values(ascending=False).head(10)
topwords = topwords_df.index.tolist()

print(f"Doc {doc_index} Topwords:\n{'='*50}")
print(topwords_df)

Doc 0 Topwords:
what       0.285714
you        0.285714
on         0.285714
koresh     0.285714
timmons    0.142857
there      0.142857
have       0.142857
he         0.142857
those      0.142857
though     0.142857
Name: 0, dtype: float64


### Modify Word Doc Distr with Word Word Distr

In [72]:
wdf_train_x = 0 * wdf_train_original
wdf_test_x = 0 * wdf_test_original
    
for wx, wo in [(wdf_train_x, wdf_train_original), (wdf_test_x, wdf_test_original)]:
    for doc_index in tqdm(range(len(wo))):
        denom = 0

        indices = (wo.loc[doc_index] > 0)
        xv = wo.loc[doc_index][indices]

        for index, (wordx, word_pr) in enumerate(xv.iteritems()):
            denom += word_pr
            wx.loc[doc_index] += word_pr * word_word_pr_distr.loc[wordx]

        wx.loc[doc_index] /= denom
    wx /= wx.max().max()

In [73]:
topwords_df = wdf_train_x.loc[doc_index].sort_values(ascending=False).head(10)
topwords = topwords_df.index.tolist()

print(f"Doc {doc_index} Topwords:\n{'='*50}")
print(topwords_df)

Doc 47 Topwords:
be      0.977938
to      0.790776
and     0.762475
of      0.759549
you     0.751423
in      0.685758
not     0.651599
for     0.630663
your    0.579761
get     0.579271
Name: 47, dtype: float64


### Modify Word Doc Distr with Word Word Distr Prime

In [74]:
wdf_train_prime = 0 * wdf_train_original
wdf_test_prime = 0 * wdf_test_original
    
for wx, wo in [(wdf_train_prime, wdf_train_original), (wdf_test_prime, wdf_test_original)]:
    for doc_index in tqdm(range(len(wo))):
        denom = 0

        indices = (wo.loc[doc_index] > 0)
        xv = wo.loc[doc_index][indices]

        for index, (wordx, word_pr) in enumerate(xv.iteritems()):
            denom += word_pr
            wx.loc[doc_index] += word_pr * word_word_pr_distr_prime.loc[wordx]

        wx.loc[doc_index] /= denom
    wx /= wx.max().max()

In [75]:
doc_index = 0

topwords_prime_df = wdf_train_prime.loc[doc_index].sort_values(ascending=False).head(10)
topwords_prime = topwords_prime_df.index.tolist()

print(f"Doc {doc_index} Topwords:\n{'='*50}")
print(topwords_prime_df)

Doc 0 Topwords:
bake       0.463731
rely       0.463731
beef       0.463731
timmons    0.463731
wasn       0.463731
fetesh     0.463731
fetish     0.463731
sic        0.463731
iii        0.463731
jerky      0.463731
Name: 0, dtype: float64


In [76]:
li = {
    "against topwords" : topwords, 
    "against topwords prime": topwords_prime
}

for title, other_words in li.items():
    print(f'{" ".join([f"doc {doc_index}", title]):^40s}')
    print("===============================")
    print("word            | wdpdo | wdpdx | wdpd_prime")
    print("-------------------------------")
    
    for other_word in other_words:
        r1 = wdf_train_original.loc[doc_index][other_word] / wdf_train_original.loc[doc_index].max()
        r2 = wdf_train_x.loc[doc_index][other_word] / wdf_train_x.loc[doc_index].max()
        r3 = wdf_train_prime.loc[doc_index][other_word] / wdf_train_prime.loc[doc_index].max()

        print(f"{other_word:15s} | {r1:5.2f} | {r2:5.2f} | {r3:5.2f}")
    print()

         doc 0 against topwords         
word            | wdpdo | wdpdx | wdpd_prime
-------------------------------
be              |  0.00 |  0.72 |  0.11
to              |  0.50 |  1.00 |  0.17
and             |  0.50 |  0.96 |  0.15
of              |  0.00 |  0.39 |  0.06
you             |  1.00 |  0.87 |  0.18
in              |  0.00 |  0.29 |  0.05
not             |  0.00 |  0.24 |  0.06
for             |  0.50 |  0.75 |  0.17
your            |  0.00 |  0.12 |  0.04
get             |  0.00 |  0.11 |  0.03

      doc 0 against topwords prime      
word            | wdpdo | wdpdx | wdpd_prime
-------------------------------
bake            |  0.50 |  0.54 |  1.00
rely            |  0.50 |  0.54 |  1.00
beef            |  0.50 |  0.54 |  1.00
timmons         |  0.50 |  0.54 |  1.00
wasn            |  0.50 |  0.54 |  1.00
fetesh          |  0.50 |  0.54 |  1.00
fetish          |  0.50 |  0.54 |  1.00
sic             |  0.50 |  0.54 |  1.00
iii             |  0.50 |  0.54 |  1.00
jer

In [77]:
wxs = [
    ("wdf_train", wdf_train),
    ("wdf_train_original", wdf_train_original),
    ("wdf_train_x", wdf_train_x),
    ("wdf_train_prime", wdf_train_prime)
]

for i in range(2, 7):
    print(f"using {i} classes")
    
    for title, wx in wxs:
        kmeans = KMeans(n_clusters=i, random_state=0).fit(wx)
        
        print(title, kmeans.inertia_)
        for ii in range(i):
            print(Counter(y_train[kmeans.labels_ == ii]))
        print()

using 2 classes
wdf_train 4484.404970760236
Counter({'sci.space': 11, 'rec.autos': 10, 'talk.politics.mideast': 9, 'alt.atheism': 6})
Counter({'talk.politics.mideast': 23, 'alt.atheism': 22, 'rec.autos': 17, 'sci.space': 14})
Counter()
Counter()

wdf_train_original 91.51846879102533
Counter({'sci.space': 11, 'rec.autos': 10, 'talk.politics.mideast': 9, 'alt.atheism': 6})
Counter({'talk.politics.mideast': 23, 'alt.atheism': 22, 'rec.autos': 17, 'sci.space': 14})
Counter()
Counter()

wdf_train_x 559.5240936910412
Counter({'talk.politics.mideast': 31, 'alt.atheism': 28, 'rec.autos': 27, 'sci.space': 25})
Counter({'talk.politics.mideast': 1})
Counter()
Counter()

wdf_train_prime 501.86225758604144
Counter({'talk.politics.mideast': 32, 'alt.atheism': 28, 'rec.autos': 27, 'sci.space': 24})
Counter({'sci.space': 1})
Counter()
Counter()

using 3 classes
wdf_train 4344.496825396824
Counter({'rec.autos': 10, 'sci.space': 8, 'talk.politics.mideast': 7, 'alt.atheism': 5})
Counter({'alt.atheism': 1